## **Создание модели классфикации заказов**
### Прежде всего, необходимо научиться классифицировать заказы, чтобы автоматически различать новые поступающие заказы

In [2]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import bokeh.models as bm
import bokeh.plotting as pl
from bokeh.io import output_notebook
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

### **Разметка датасета**
Перед разработкой модели классфикации разметим имеющиеся данные:
- Использование эмбеддингов для получения векторного представления текстов заказов
- Применение метода кластеризации, для объединения заказов в группы

#### **Шаг 1**: Загрузка данных

In [3]:
cases = pd.read_csv('../data/cases.csv', sep=';')
print(cases.columns.to_list())
cases.rename(columns={'\ufeffНомер поручения':'Номер поручения'}, inplace=True)
cases.head()

['\ufeffНомер поручения', 'Заказчик', 'Дата поручения', 'Выполнено', 'Дата выполнения', 'Затрачено дней', 'Сумма вознаграждения', 'Описание']


,Номер поручения,Заказчик,Дата поручения,Выполнено,Дата выполнения,Затрачено дней,Сумма вознаграждения,Описание
0,11000,Анна,1051-08-11,да,1051-08-21,4.0,6000,В лесу по дороге от пещеры звери нападают на л...
1,11001,Мария,1051-07-09,да,1051-09-02,2.0,20000,В лесу по дороге от пещеры были замечены разбо...
2,11002,Эмилио,1053-10-05,да,1053-10-20,7.0,22500,По дороге из деревни монстры похитили путников...
3,11003,Бьянка,1052-11-24,да,1052-12-02,5.0,5500,Недалеко от города видели монстров. Нужно побе...
4,11004,Бьянка,1052-02-23,да,1052-03-30,8.0,10500,В деревне у меня пропала сумка с документами. ...


#### **Шаг 2**: Инициализация модели трансформера для получения эмбеддингов

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# Получаем эмбеддинги для предложений
embeddings = model.encode(cases['Описание'])

embeddings.shape

(500, 384)

#### Просмотрев тексты заказов, можно выделить 4 основные группы:
- Заказ на чудовище в пещере,
- Спасение путников,
- Поиск пропажи,
- где-то завелись монстры/разбойники

#### **Шаг 3**: Кластеризация предложений с помощью `KMeans`

In [6]:
# Зададим количество кластеров
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, algorithm='lloyd', random_state=42, n_init=10)
kmeans.fit(embeddings)

# Получаем метки кластеров для каждого предложения
labels = kmeans.labels_

#### **Шаг 4**: Визуализация кластеров с использованием `PCA` для уменьшения размерности до 2

In [7]:
output_notebook()


def draw_vectors(
    x,
    y,
    radius=10,
    alpha=0.9,
    #color="blue",
    width=600,
    height=400,
    show=True,
    **kwargs,
):
    color = np.array([(0, 0, 0)] * len(x))
    for i in range(num_clusters):
        new_color = (np.random.randint(100), np.random.randint(100), np.random.randint(100))
        color[labels==i] = new_color
    """draws an interactive plot for data points with auxilirary info on hover"""
    data_source = bm.ColumnDataSource({"x": x, "y": y, "color": color.tolist(), **kwargs})

    fig = pl.figure(active_scroll="wheel_zoom", width=width, height=height)
    fig.scatter("x", "y", size=radius, color="color", alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig

Loading BokehJS ...

In [8]:
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

In [13]:
draw_vectors(reduced_embeddings[:, 0], reduced_embeddings[:, 1], token=cases['Номер поручения'].astype(str) + ' ' + cases['Описание'])

figure(id='p1208', ...)

Видно хорошее разделение на 3 кластера, но в класс со спасением путников залезли объекты из другого класса

In [14]:
# номера заказов, которые были неверно отнесены методом кластеризации в первый класс
missclass = [11202, 11122, 11016, 11226, 11482, 11431, 11193, 11003]
labels[cases.loc[cases['Номер поручения'].isin(missclass)].index]

array([1, 1, 1, 1, 1, 1, 1, 1])

In [20]:
labels[cases.loc[cases['Номер поручения'].isin(missclass)].index] = 3
draw_vectors(reduced_embeddings[:, 0], reduced_embeddings[:, 1], token=cases['Номер поручения'].astype(str) + ' ' + cases['Описание'])

figure(id='p1514', ...)

#### **Шаг 5**: Добавление меток в датасет, сохранение обработанного датасета

In [21]:
cases['Тип поручения'] = labels
cases.to_csv('../data/cases_labeled.csv', index=False, sep=';')